## 1. 디렉토리, 라이브러리 설정, 데이터 불러오기

In [ ]:
from google import colab
colab.drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/Shareddrives/방세 1팀/data/"
SEED = 316

In [ ]:
import random
random.seed(SEED)
import numpy as np, pandas as pd, os
np.random.seed(SEED)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score

In [ ]:
train = pd.read_csv(f'{DATA_PATH}train.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')

## 2. Prediction

In [ ]:
#모델링 과정에서 grid search를 통해 각 모델별 최적의 parameter를 best_params에 저장 
best_params = [0.2, 0.1, 0.1, 0.1, 0.1, 0.3, 0.2, 0.1, 0.1, 0.1, 0.4, 0.4, 0.1,
       0.2, 0.5, 0.1, 0.5, 0.5, 0.5, 0.1, 0.1, 0.1, 0.1, 0.3, 0.1, 0.4,
       0.2, 0.4, 0.4, 0.1, 0.2, 0.1]

cols = [c for c in train.columns if c not in ['id', 'target', 'magic']]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
preds = np.zeros(len(test))
selector = VarianceThreshold(1.5)

# 각 magic 카테고리에 따라 32개의 모델을 구성
for i in range(32): 
    train_by_m = train[train.magic == i]
    test_by_m = test[test.magic == i]
    idx1 = train_by_m.index; idx2 = test_by_m.index
    train_by_m.reset_index(drop=True,inplace=True)
    test_by_m.reset_index(drop=True,inplace=True)
    
    # Column 별 분산에 따라 feature 선택 (분산 지수가 1.5 이하인 것들은 제외)
    train_selected = selector.fit_transform(train_by_m[cols])
    X_train = train_selected[:train_by_m.shape[0]];
    # 앞 train set에서 선택된 column을 test에서 그대로 선택
    X_test = test_by_m[cols].loc[:, selector.get_support()]

    # 모델 하나 당(magic 카테고리 하나 당), 5-fold 만들어 평가 및 예측 
    skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
    for tr_idx, val_idx in skf.split(train_by_m, train_by_m.target):
        clf = QDA(reg_param = best_params[i])
        # 훈련은 X_train의 tr_idx로 진행
        clf.fit(X_train[tr_idx,:], train_by_m.loc[tr_idx].target)
        # 5개 예측값들의 평균을 도출 
        preds[idx2] += clf.predict_proba(X_test)[:,1] / skf.n_splits

In [ ]:
preds = np.where(preds >= 0.5 , 1, 0)

1. predict_proba 는 2개의 컬럼을 만듭니다. (target이 0이 될 확률, target이 1이 될 확률)
2. 따라서 threshold (default: 0.5)를 설정해서 target이 integer로 나누어지도록 해야합니다.
3. 만약, target이 unbalanced 하다면, auc_roc_curve를 그림으로써 최적의 threshold를 찾아야 하나, 여기서는 target간 비율이 balanced하기 때문에 디폴트 threshold 값 0.5를 사용하였습니다. 

## 3.Submission

In [ ]:
submission = pd.read_csv(f'{DATA_PATH}제출/submission.csv')
submission['target'] = preds
submission.to_csv(f'{DATA_PATH}제출/submission_방세 1팀.csv',index=False)